In [66]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, GlobalAveragePooling2D, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Add, add
from tensorflow.keras.layers import MaxPooling2D, ZeroPadding2D

In [67]:
def first_conv(input_tensor):
    # 7*7 Conv 연산을 수행하며, feature map을 생성해야 함. input_tensor 크기의 절반으로 생성하기
    x = ZeroPadding2D(padding = (3, 3), name = 'Conv1_Zero')(input_tensor)
    x = Conv2D(64, kernel_size = (7, 7), strides = (2, 2), padding = 'valid', kernel_initializer = 'he_normal', name = 'Conv1_Conv')(x)
    x = BatchNormalization(axis = 3, name = 'Conv1_batch')(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding = (1, 1), name = 'Conv1_Zero2')(x)
    x = MaxPooling2D((3, 3), strides = (2, 2), name = 'Conv1_Max')(x)
    
    return x

In [76]:
def conv_block(input_tensor, middle_kernel_size, filters, stage, block, strides = (2, 2)):
    filter1, filter2, filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = Conv2D(filters = filter1, kernel_size = (1, 1), strides = strides, kernel_initializer = 'he_normal', name = conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis = 3, name = bn_name_base +'2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters = filter2, kernel_size = middle_kernel_size, padding = 'same', kernel_initializer = 'he_normal', name = conv_name_base + '2b')(x)
    x = BatchNormalization(axis = 3, name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters = filter3, kernel_size = middle_kernel_size, padding = 'same', kernel_initializer = 'he_normal', name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis = 3, name = bn_name_base + '2c')(x)
    
    shortcut = Conv2D(filters = filter3, kernel_size = (1, 1), strides = strides, kernel_initializer = 'he_normal', name = conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(shortcut)
    
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    
    return x

In [77]:
def identity_block(input_tensor, middle_kernel_size, filters, stage, block):
    filter1, filter2, filter3 = filters
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = Conv2D(filters = filter1, kernel_size = (1, 1), kernel_initializer = 'he_normal', name = conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis = 3, name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters = filter2, kernel_size = middle_kernel_size, padding = 'same', kernel_initializer = 'he_normal', name = conv_name_base + '2b')(x)
    x = BatchNormalization(axis = 3, name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters = filter3, kernel_size = (1, 1), kernel_initializer = 'he_normal', name = conv_name_base + '2c')(x)
    x = BatchNormalization(axis = 3, name = bn_name_base + '2c')(x)
    
    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    
    return x

In [78]:
def Create_ResNet(input_shape = (224 , 224, 3), n_classes = 10):
    input_tensor = Input(input_shape)
    
    x = first_conv(input_tensor)
    
    x = conv_block(x, middle_kernel_size=(3, 3), filters = [64, 64 ,256], stage = 2, block= 'a', strides = (1, 1))
    x = identity_block(x, middle_kernel_size = (3, 3), filters= [64, 64, 256], stage = 2, block = 'b')
    x = identity_block(x, middle_kernel_size= (3, 3), filters= [64, 64, 256], stage = 2, block = 'c')
    
    x = conv_block(x, middle_kernel_size = (3, 3), filters= [128, 128, 512], stage = 3, block = 'a')
    x = identity_block(x, middle_kernel_size = (3, 3), filters= [128, 128, 512], stage = 3, block = 'b')
    x = identity_block(x, middle_kernel_size = (3, 3), filters= [128, 128, 512], stage = 3, block = 'c')
    x = identity_block(x, middle_kernel_size = (3, 3), filters= [128, 128, 512], stage = 3, block = 'd')
    
    x = conv_block(x, middle_kernel_size = (3, 3), filters= [256, 256, 1024], stage = 4, block = 'a')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [256, 256, 1024], stage = 4, block = 'b')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [256, 256, 1024], stage = 4, block = 'c')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [256, 256, 1024], stage = 4, block = 'd')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [256, 256, 1024], stage = 4, block = 'e')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [256, 256, 1024], stage = 4, block = 'f')
    
    x = conv_block(x, middle_kernel_size = (3, 3), filters= [512, 512, 2048], stage = 5, block = 'a')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [512, 512, 2048], stage = 5, block = 'b')
    x = identity_block(x, middle_kernel_size = (3, 3), filters = [512, 512, 2048], stage = 5, block = 'c')
    
    x = GlobalAveragePooling2D(name = 'G_A_Pooling')(x)
    output = Dense(n_classes, activation = 'softmax', name = 'outputs')(x)
    
    model = Model(inputs = input_tensor, outputs = output, name = 'ResNet50')
    model.summary()
    
    return model

In [79]:
model = Create_ResNet(input_shape=(224,224,3), n_classes=10)

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_Zero (ZeroPadding2D)      (None, 230, 230, 3)  0           input_13[0][0]                   
__________________________________________________________________________________________________
Conv1_Conv (Conv2D)             (None, 112, 112, 64) 9472        Conv1_Zero[0][0]                 
__________________________________________________________________________________________________
Conv1_batch (BatchNormalization (None, 112, 112, 64) 256         Conv1_Conv[0][0]                 
___________________________________________________________________________________________

In [80]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

In [82]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
import cv2
import sklearn


def zero_one_scaler(image):
    return image/255.0

def get_preprocessed_ohe(images, labels, pre_func=None):
   
    if pre_func is not None:
        images = pre_func(images)
    
    oh_labels = to_categorical(labels)
    return images, oh_labels

def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):

    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels )

In [84]:
class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size = BATCH_SIZE, augmentor = None, shuffle = False, pre_func = None):
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.shuffle = shuffle
        self.pre_func = pre_func
        
        if self.shuffle:
            self.on_epoch_end()
            pass
        
    def __len__(self):
        return int(np.ceil(len(self.labels) / self.batch_size))

    def __getitem__(self, index):
       
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        image_batch = np.zeros((images_fetch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
    
        for image_index in range(images_fetch.shape[0]):
          
            image = cv2.resize(images_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))
          
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
                
          
            if self.pre_func is not None:
                image = self.pre_func(image)
            
          
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if(self.shuffle):
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass    

In [85]:
# CIFAR10 데이터 재 로딩 및 OHE 전처리 적용하여 학습/검증/데이터 세트 생성. 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(40000, 32, 32, 3) (40000, 10) (10000, 32, 32, 3) (10000, 10) (10000, 32, 32, 3) (10000, 10)


In [88]:
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

tr_ds = CIFAR_Dataset(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=True, pre_func=resnet_preprocess)
val_ds = CIFAR_Dataset(val_images, val_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)

In [91]:
resnet_model = Create_ResNet(input_shape=(128, 128, 3), n_classes=10)

resnet_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 5번 iteration내에 validation loss가 향상되지 않으면 learning rate을 기존 learning rate * 0.2로 줄임.  
rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

history = resnet_model.fit(tr_ds, epochs=30, 
                    validation_data=val_ds, 
                    callbacks=[rlr_cb, ely_cb]
                   )

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_Zero (ZeroPadding2D)      (None, 134, 134, 3)  0           input_14[0][0]                   
__________________________________________________________________________________________________
Conv1_Conv (Conv2D)             (None, 64, 64, 64)   9472        Conv1_Zero[0][0]                 
__________________________________________________________________________________________________
Conv1_batch (BatchNormalization (None, 64, 64, 64)   256         Conv1_Conv[0][0]                 
___________________________________________________________________________________________

C:\Users\c\.conda\envs\wongi\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


625/625 [==============================] - 93s 133ms/step - loss: 1.5688 - accuracy: 0.4596 - val_loss: 2.2518 - val_accuracy: 0.3919
Epoch 2/30
625/625 [==============================] - 82s 131ms/step - loss: 1.0398 - accuracy: 0.6352 - val_loss: 1.3387 - val_accuracy: 0.5690
Epoch 3/30
625/625 [==============================] - 82s 131ms/step - loss: 0.7778 - accuracy: 0.7276 - val_loss: 1.1344 - val_accuracy: 0.6603
Epoch 4/30
625/625 [==============================] - 82s 131ms/step - loss: 0.6178 - accuracy: 0.7830 - val_loss: 1.1053 - val_accuracy: 0.6556
Epoch 5/30
625/625 [==============================] - 82s 131ms/step - loss: 0.5152 - accuracy: 0.8206 - val_loss: 0.7716 - val_accuracy: 0.7497
Epoch 6/30
625/625 [==============================] - 82s 131ms/step - loss: 0.4129 - accuracy: 0.8566 - val_loss: 1.0583 - val_accuracy: 0.6989
Epoch 7/30
625/625 [==============================] - 84s 135ms/step - loss: 0.3290 - accuracy: 0.8858 - val_loss: 0.7535 - val_accuracy: 0.7

In [92]:
test_ds = CIFAR_Dataset(test_images, test_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)
resnet_model.evaluate(test_ds)

157/157 [==============================] - 6s 41ms/step - loss: 0.8521 - accuracy: 0.8541


[0.8521357178688049, 0.8540999889373779]